<a href="https://colab.research.google.com/github/KeerthanaAjith/Machine-Learning-Assignments/blob/main/Lab_8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Keerthana Ajith
# BL.EN.U4CSE21098
# CSE B

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/Book1.csv')
df

,age,income,student,credit rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31 . . .40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31 . . .40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [4]:
# A1.

data = [
    ['<=30', 'high', 'no', 'fair', 'no'],
    ['<=30', 'high', 'no', 'excellent', 'no'],
    ['31 . . .40', 'high', 'no', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'excellent', 'no'],
    ['31 . . .40', 'low', 'yes', 'excellent', 'yes'],
    ['<=30', 'medium', 'no', 'fair', 'no'],
    ['<=30', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'yes', 'fair', 'yes'],
    ['<=30', 'medium', 'yes', 'excellent', 'yes'],
    ['31 . . .40', 'medium', 'no', 'excellent', 'yes'],
    ['31 . . .40', 'high', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'excellent', 'no']
]

# Extracting the "buys_computer" column
buys_computer_col = [row[-1] for row in data]

# Calculating the prior probabilities
total_inst = len(buys_computer_col)
buys_computer_no = buys_computer_col.count('no')
buys_computer_yes = buys_computer_col.count('yes')

# Calculating the prior probabilities
prior_no = buys_computer_no / total_inst
prior_yes = buys_computer_yes / total_inst

print(f'Prior probability for "buys_computer = no": {prior_no:.3f}')
print(f'Prior probability for "buys_computer = yes": {prior_yes:.3f}')

Prior probability for "buys_computer = no": 0.357
Prior probability for "buys_computer = yes": 0.643


In [5]:
# A2.

# Calculating class-conditional probabilities
def calculate_conditional_probability(feature, label, df):
    conditional_probs = {}
    classes = df[label].unique()

    for c in classes:
        class_data = df[df[label] == c]
        conditional_probs[c] = class_data[feature].value_counts(normalize=True).to_dict()

    return conditional_probs

# Calculate class-conditional probabilities for each feature
for feature in df.columns[:-1]:  # Exclude the target variable
    conditional_probs = calculate_conditional_probability(feature, 'buys_computer', df)

    print(f"Class-conditional densities for {feature}:")
    for label, probs in conditional_probs.items():
        print(f"P({feature} | buys_computer = {label}):")
        for value, prob in probs.items():
            print(f"  P({feature} = {value} | buys_computer = {label}): {prob:.4f}")
        print()

Class-conditional densities for age:
P(age | buys_computer = no):
  P(age = <=30 | buys_computer = no): 0.6000
  P(age = >40 | buys_computer = no): 0.4000

P(age | buys_computer = yes):
  P(age = 31 . . .40 | buys_computer = yes): 0.4444
  P(age = >40 | buys_computer = yes): 0.3333
  P(age = <=30 | buys_computer = yes): 0.2222

Class-conditional densities for income:
P(income | buys_computer = no):
  P(income = high | buys_computer = no): 0.4000
  P(income = medium | buys_computer = no): 0.4000
  P(income = low | buys_computer = no): 0.2000

P(income | buys_computer = yes):
  P(income = medium | buys_computer = yes): 0.4444
  P(income = low | buys_computer = yes): 0.3333
  P(income = high | buys_computer = yes): 0.2222

Class-conditional densities for student:
P(student | buys_computer = no):
  P(student = no | buys_computer = no): 0.8000
  P(student = yes | buys_computer = no): 0.2000

P(student | buys_computer = yes):
  P(student = yes | buys_computer = yes): 0.6667
  P(student = no 

In [6]:
# A3.

from scipy.stats import chi2_contingency

# Creating a contingency table
contingency_tb = pd.crosstab(index=df['age'], columns=[df['income'], df['student'], df['credit rating']])

# Performing chi-square test
chi2, p, _, _ = chi2_contingency(contingency_tb)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

# Interpreting the results
alpha = 0.05
print(f"Significance level: {alpha}")
print(f"Is the p-value less than alpha? {'Yes' if p < alpha else 'No'}")

if p < alpha:
    print("Is rejecting the null hypothesis. There is evidence of a significant association between the features.")
else:
    print("Is failing to reject the null hypothesis. There is no significant evidence of an association between the features.")

Chi-square statistic: 12.95
P-value: 0.6764100579553458
Significance level: 0.05
Is the p-value less than alpha? No
Is failing to reject the null hypothesis. There is no significant evidence of an association between the features.


In [7]:
# A4.

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Encoding categorical variables
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

# Spliting the data into training and testing sets
X = df.drop('buys_computer', axis=1)
y = df['buys_computer']
Tr_X, Te_X, Tr_y, Te_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Building and training the Naïve-Bayes classifier
mod = GaussianNB()
mod.fit(Tr_X, Tr_y)

# Making predictions on the test set
predictions = mod.predict(Te_X)

# Evaluation of the model
accuracy = accuracy_score(Te_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Displaying classification report
print("Classification Report:")
print(classification_report(Te_y, predictions))

Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# A5.
# Our project utilises and is working on images so, building NB classifier is not possible.